In [49]:
import pandas
import requests
import datetime

In [2]:
etherscan_api_key = "SU775DFF8GEGMER3B8ZPK3QZIJWGIBE45T"

In [13]:
##https://api.etherscan.io/api?module=block&action=getblockreward&blockno=2165403&apikey=YourApiKeyToken
url_endpoint = "https://api.etherscan.io/api"
params = {"module" : "block", "action" : "getblockreward", "blockno" : "2165403", "apikey" : etherscan_api_key }
resp = requests.get(url_endpoint, params=params)

In [31]:
##Getting the most recent block number
params = {"module": "proxy", "action":"eth_blockNumber", "apikey": etherscan_api_key}
resp = requests.get(url_endpoint, params=params)
resp_dict = eval(resp.content) #Get a string, must eval to a dict
latest_block = int(resp_dict['result'], 0) #Get it in Hex, need to eval to int
latest_block

4836665

In [35]:
#Starting Point
starting_block = 4830000 #picked a solid starting point
hex_starting_block = hex(starting_block)
hex_starting_block

'0x49b330'

In [37]:
#Get Transactions in a Block
#https://github.com/ethereum/wiki/wiki/JSON-RPC#eth_gettransactionbyhash
params = {"module":"proxy", "action":"eth_getBlockByNumber", "tag" : hex_starting_block, "boolean":"true", "apikey": etherscan_api_key}
resp = requests.get(url_endpoint, params=params)
resp_dict = eval(resp.content) #Get a string, must eval to a dict

{'id': 1,
 'jsonrpc': '2.0',
 'result': {'difficulty': '0x6b6e3f4f1788d',
  'extraData': '0xd5830107068650617269747986312e32302e30826c69',
  'gasLimit': '0x79e349',
  'gasUsed': '0x1097db',
  'hash': '0x59a0c39d5627266d28acd8135d66e017241497eb3a95dbe7fc75968e130f90af',
  'logsBloom': '0x00000000008000000208200000001000004000000000000000000000000000804000000800000000010401001000020020000000000080000020000000000800000000040001000000000008000010000400000000002080008008000000000010000000540000020010400020000000002000020000000004000110000000000040801000220018800020000000500000001400808c0000000000000000000000000000000000000000020010020000208000000041000200020000000020000000000600001000000000000000000000000000000002000000000804000000000000800000004010000000000020e010010200000200048000104000000000',
  'miner': '0x6a7a43be33ba930fe58f34e07d0ad6ba7adb9b1f',
  'mixHash': '0x0cfcb75e1d10cf3f398cd36c24701208ded37f6523ed9d36f9496dc173f2fe17',
  'nonce': '0x34e34765af2dfd3b',
  'number': '0x49b330',


In [51]:
#Datapoints per transaction needed
# Timestamp
# Account Sending
# Account Receiving (May be smart contract)
# Tx Fee
# Size of Date
# Network Name (duh)
# Transaction Status

timestamp = int(resp_dict['result']['timestamp'],0)
timestamp = datetime.datetime.fromtimestamp(timestamp).strftime("%Y-%m-%d %H:%M:%S")
timestamp

'2017-12-31 07:59:48'